# PROJETO OBSERVATÓRIO FIEG #

### Em busca das melhores alternativas para orientação estratégica e oportunidades o OBSERVATÓRIO FIEG é um transformador de realidade para o mercado, pois presta serviços de análises estratégicas para subsidiar tomadas de decisões.

O Observatório fieg trabalha para identificar os comportamentos do Mercado apresentando um diagnóstico confiável, através de dados internos, externos e pesquisas, que auxiliem a tomada de decisão. Foi uma das maiores empresas que trabalhei na parte de engenharia de dados.

### Um de meus desafios ###
 Um de meus objetivos era pegar a base de dados do CAGED que é a sigla para Cadastro Geral de Empregados e Desempregados (CAGED). Eu tinha que baixar sua base de dados e coloca-las em uma planilha de excel e extrair os dados, trata-los e envia-los para o Github do observatório para que ele possa gerar uma API, para que possamos coloca-lo no wordpress e mostrar os dados do saldo de empregados e desempregados na pagina web  do sistema do observatório que inclusive você pode encontra-lo aqui : https://observatoriofieg.com.br/panorama-economico/ , crie sua conta e veja os cards, para cada um dos card foi feito um script python para extrair, tratar e enviar estes dados para o github do obs para que ele possa mostrar estes dados nesta pagina. 
 Github do Observatório : https://github.com/observatoriofieg
 
 
 # Veja abaixo o script que fiz para o card do saldo empregos e desempregados da industria. encontrado aqui:
 ⬇️
 https://observatoriofieg.com.br/panorama-economico/
 
 
 


In [1]:

#BIBLIOTECAS NECESSÁRIAS
from cgi import print_arguments
from sys import displayhook
import pandas as pd
import json
from upload import *
import util as utils
from util import *
from github import Github # Esse import do github é nosso do observatório, não posso divulga-lo, por esse motivo o script aqui não funciona!
from datetime import datetime
from openpyxl import load_workbook
import locale

#Acessando planilha com os dados do caged
table_caged = pd.read_excel("saldo-caged-industria-da-transformacao-pablo.xlsx")
df = pd.DataFrame(table_caged.values)
df.columns = df.iloc[0]
df = df.iloc[1:]
df.columns = ['referencia', 'indice_br','indice_go']
df['referencia'] =  pd.to_datetime(df['referencia'],  format='%d%b%Y:%H:%M:%S.%f')
df.sort_values('referencia', inplace =True) 
df = df.reset_index(drop=True)
df.dropna(inplace =True)

df['referencia'] = df['referencia'].astype(str)
df['referencia'] = df['referencia'].str[:7]
get_colums = table_caged[['referencia', 'Brasil']]
print(get_colums)

# parte responsável pelo mês.
meses_ano = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
ano_referencia = df['referencia'][-1:].iloc[0][:4]
mes_referencia = int(df['referencia'][-1:].iloc[0][-2:])
referencia = meses_ano[mes_referencia-1] + '/' + ano_referencia

#extraindo o ultimo dado de brasil
print("agora...fazendo a extração ultimo dado de brasil...")
ultimos_dado_br = pd.read_excel("saldo-caged-industria-da-transformacao-pablo.xlsx")
ultimo_dado_br = ultimos_dado_br.loc[0,'Brasil']
print(ultimo_dado_br)
print("Ultimo dado de brasil extraido com sucesso!")


#extraindo o dado de goiás 
ultimos_dado_go = pd.read_excel("saldo-caged-industria-da-transformacao-pablo.xlsx")
ultimo_dado_go = ultimos_dado_go.loc[0,'Goiás']
print(ultimo_dado_go)
print("Ultimo dado de Goiás extraído com sucesso!")

#Criando série histórica
print("Criando série histórica...")

serie_br = []
serie_go= []
now = datetime.now()
ano_atual = now.year
ano_anterior = ano_atual - 1

for i, row in df.iterrows():
  mes = row[0][:7].replace('-', '')
  mes = pd.to_datetime(mes, format='%Y%m', errors='coerce')
  mes = mes.strftime("%Y-%m-%dT%H:%M:%SZ")
  serie_go.append({'x': mes, 'y': row[1]})
  serie_br.append({'x': mes, 'y': row[2]})
 
print('- Série criada')
print("Iniciando o condicionamento!")

if str(ultimos_dado_br) < '0':
    direcao_br = 'down'
    cor_br = 'red'
   
else:
    direcao_br = 'up'
    cor_br = 'green'

if str(ultimos_dado_go) < '0' :
    direcao_go = 'down'
    cor_go = 'red'

else:
    direcao_go = 'up'
    cor_go = 'green'

print('- Valores do cartão foram armazenados')


# Json caged template

caged_json = {

    'nome': 'Saldo de Empregados da Ind. de Transformação',
    'descricao': 'Saldo de empregados = Admitidos - Desligados',
    'fonte': 'MTE',
    'stats': [
        {
            'titulo': 'Brasil',
            'valor': str(ultimo_dado_br), 
            'direcao': direcao_br,
            'cor_valor': cor_br,
            'referencia': referencia,
            'desc_serie': 'Saldo de empregados por mês',
            'serie_tipo': 'data',
            'y_label': {
                'prefixo_sufixo': 'sufixo',
                'label': '',
            },


            'serie_labels': {
                'x': 'Data',
                'y': 'Valor'
            },

            'serie': serie_go, 
        },

        {
            'titulo': 'Goiás',
            'valor': str(ultimo_dado_go),
            'direcao': direcao_go,
            'cor_valor': cor_go,
            'referencia': referencia,
            'desc_serie': 'Saldo de empregados por mês',
            'serie_tipo': 'data',
            'y_label': {
                'prefixo_sufixo': 'sufixo',
                'label': '',
            },

            'serie_labels': {
                'x': 'Data',
                'y': 'Valor'
            },
            'serie': serie_br, 
        },
    ]
  }

print(caged_json)


'''
 OBS : Crie uma conta para acessar o panorama econômico, 
depois disso caçe o card chamado : "Saldo de Empregados da Ind. de Transformação e veja os dados lá." 
'''

# Enviando os dados para o github para gerar um API para o site do obsevatório : https://observatoriofieg.com.br/panorama-economico
path_save_json = util.config['path_save_json']['path']
name_json = 'saldo_desemprego_caged'
with open(path_save_json + name_json + '.json', 'w', encoding='utf-8') as f:
 json.dump(caged_json, f, indent=4, sort_keys=True, default=str)
print('- JSON armazenado')
upload_files_to_github(name_json)
print('- JSON enviado para o GitHub')
print('- Saldo de Empregados da Ind. de Transformação atualizado com sucesso!')

ModuleNotFoundError: No module named 'upload'